In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_columns=1000 
pd.options.display.max_rows=1000
import numpy as np

In [2]:
import sys
sys.path.append('ml_modules/')

In [3]:
DATA_DIR = 'C:/Users/arvin/Springboard/Capstone-3/AMexpert/train_AUpWtIz/'
DW_OUTPUTS='C:/Users/arvin/Springboard/Capstone-3/AMexpert/train_AUpWtIz/DW_OUTPUTS/'

train = pd.read_csv(DATA_DIR + 'train.csv')
"train", train.shape

test = pd.read_csv(DATA_DIR + 'test_QyjYwdj.csv')
"test", test.shape

campaign_data = pd.read_csv(DATA_DIR + 'campaign_data.csv')
campaign_data['start_date'] = pd.to_datetime(campaign_data.start_date, format='%d/%m/%y')
campaign_data['end_date'] = pd.to_datetime(campaign_data.end_date,  format='%d/%m/%y')
campaign_data['duration'] = (campaign_data.end_date - campaign_data.start_date).dt.days

('train', (78369, 5))

('test', (50226, 4))

Merge Features

In [4]:
cust_hist_trans = pd.read_csv(DW_OUTPUTS +'cust_hist_trans.csv')
cust_hist_trans['start_date'] = pd.to_datetime(cust_hist_trans.start_date)

cust_hist_trans_daily = pd.read_csv(DW_OUTPUTS +'cust_hist_trans_daily.csv')
cust_hist_trans_daily['start_date'] = pd.to_datetime(cust_hist_trans_daily.start_date)

coup_hist_trans = pd.read_csv(DW_OUTPUTS +'coup_hist_trans.csv')
coup_hist_trans['start_date'] = pd.to_datetime(coup_hist_trans.start_date)

coup_hist_trans_daily = pd.read_csv(DW_OUTPUTS +'coup_hist_trans_daily.csv')
coup_hist_trans_daily['start_date'] = pd.to_datetime(coup_hist_trans_daily.start_date)

cust_coup_hist_trans = pd.read_csv(DW_OUTPUTS +'cust_coup_hist_trans.csv')
cust_coup_hist_trans['start_date'] = pd.to_datetime(cust_coup_hist_trans.start_date)

cust_coup_hist_trans_daily = pd.read_csv(DW_OUTPUTS +'cust_coup_hist_trans_daily.csv')
cust_coup_hist_trans_daily['start_date'] = pd.to_datetime(cust_coup_hist_trans_daily.start_date)

coup_brand_hist_trans = pd.read_csv(DW_OUTPUTS +'coup_brand_hist_trans.csv')
coup_brand_hist_trans['start_date'] = pd.to_datetime(coup_brand_hist_trans.start_date)

cust_brand_hist_trans = pd.read_csv(DW_OUTPUTS +'cust_brand_hist_trans.csv')
cust_brand_hist_trans['start_date'] = pd.to_datetime(cust_brand_hist_trans.start_date)

cust_coup_brand_hist_trans = pd.read_csv(DW_OUTPUTS +'cust_coup_brand_hist_trans.csv')
cust_coup_brand_hist_trans['start_date'] = pd.to_datetime(cust_coup_brand_hist_trans.start_date)

In [5]:
def merge_dfs(df):
    df = df.merge(campaign_data, on='campaign_id', how='left', suffixes=['', '_camp']
                  
    ).merge(coup_hist_trans, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_coht']
    ).merge(coup_hist_trans_daily, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_cohtd']
            
    ).merge(cust_hist_trans, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cuht']
    ).merge(cust_hist_trans_daily, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cuhtd']
            
    ).merge(cust_coup_hist_trans, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucoht']
    ).merge(cust_coup_hist_trans_daily, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucohtd']
            
    ).merge(coup_brand_hist_trans, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_cobrht']
    ).merge(cust_brand_hist_trans, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cubrht']
    ).merge(cust_coup_brand_hist_trans, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucobrht']
            
    )
    df.shape
    df.head()
    return df
train.shape, test.shape
train = merge_dfs(train)
test = merge_dfs(test)

train.shape, test.shape

((78369, 5), (50226, 4))

((78369, 108), (50226, 107))

In [6]:
train.columns.tolist()

['id',
 'campaign_id',
 'coupon_id',
 'customer_id',
 'redemption_status',
 'campaign_type',
 'start_date',
 'end_date',
 'duration',
 'item_id_count',
 'selling_price_sum',
 'selling_price_mean',
 'selling_price_std',
 'other_discount_sum',
 'other_discount_mean',
 'coupon_discount_sum',
 'coupon_discount_mean',
 'coupon_applied_sum',
 'coupon_applied_mean',
 'date_count',
 'selling_price_sum_mean',
 'selling_price_sum_std',
 'selling_price_sum_last',
 'other_discount_sum_mean',
 'other_discount_sum_std',
 'other_discount_sum_last',
 'coupon_discount_sum_mean',
 'coupon_discount_sum_std',
 'coupon_discount_sum_last',
 'coupon_applied_sum_mean',
 'coupon_applied_sum_std',
 'coupon_applied_sum_last',
 'item_id_count_cuht',
 'selling_price_sum_cuht',
 'selling_price_mean_cuht',
 'selling_price_std_cuht',
 'other_discount_sum_cuht',
 'other_discount_mean_cuht',
 'coupon_discount_sum_cuht',
 'coupon_discount_mean_cuht',
 'coupon_applied_sum_cuht',
 'coupon_applied_mean_cuht',
 'date_count_

In [7]:
id_col = 'id'
target_col = 'redemption_status'

columns_to_drop = []
columns_to_drop = [c for c in columns_to_drop if c in train.columns]

cat_cols = [
    'campaign_id',
    'coupon_id',
    'customer_id',
    'campaign_type',
    'start_date',
    'end_date',    
]
cat_cols = [c for c in cat_cols if c in train.columns]
cat_cols

['campaign_id',
 'coupon_id',
 'customer_id',
 'campaign_type',
 'start_date',
 'end_date']

In [8]:
print(id_col)
print(target_col)
print(columns_to_drop)
print(cat_cols)

id
redemption_status
[]
['campaign_id', 'coupon_id', 'customer_id', 'campaign_type', 'start_date', 'end_date']


In [9]:
# imputing categorical columns
train[cat_cols] = train[cat_cols].fillna('Missing')
test[cat_cols] = test[cat_cols].fillna('Missing')

# imputing numerical columns
train = train.fillna(-1)
test = test.fillna(-1)

In [10]:
group_col = train['campaign_id'].astype(str) 
group_col_coupon = train['coupon_id'].astype(str)
train.shape
train.head()

(78369, 108)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_sum_mean,selling_price_sum_std,selling_price_sum_last,other_discount_sum_mean,other_discount_sum_std,other_discount_sum_last,coupon_discount_sum_mean,coupon_discount_sum_std,coupon_discount_sum_last,coupon_applied_sum_mean,coupon_applied_sum_std,coupon_applied_sum_last,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_sum_mean_cuhtd,selling_price_sum_std_cuhtd,selling_price_sum_last_cuhtd,other_discount_sum_mean_cuhtd,other_discount_sum_std_cuhtd,other_discount_sum_last_cuhtd,coupon_discount_sum_mean_cuhtd,coupon_discount_sum_std_cuhtd,coupon_discount_sum_last_cuhtd,coupon_applied_sum_mean_cuhtd,coupon_applied_sum_std_cuhtd,coupon_applied_sum_last_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_sum_mean_cucohtd,selling_price_sum_std_cucohtd,selling_price_sum_last_cucohtd,other_discount_sum_mean_cucohtd,other_discount_sum_std_cucohtd,other_discount_sum_last_cucohtd,coupon_discount_sum_mean_cucohtd,coupon_discount_sum_std_cucohtd,coupon_discount_sum_last_cucohtd,coupon_applied_sum_mean_cucohtd,coupon_applied_sum_std_cucohtd,coupon_applied_sum_last_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,1,13,27,1053,0,X,2013-05-19,2013-07-05,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,2,13,116,48,0,X,2013-05-19,2013-07-05,47,66.0,4717.27,71.473788,28.756222,677.15,10.259848,0.00,0.000000,0.0,0.000000,53.0,89.005094,45.728057,59.49,12.776415,10.914820,11.4,0.000000,0.000000,0.00,0.000000,0.000000,0.0,353,85211.51,241.392380,414.052660,9476.60,26.845892,1237.79,3.506487,12,0.033994,60,1420.191833,703.689159,124.67,157.943333,201.953340,0.00,20.629833,53.818250,0.0,0.200000,0.546173,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,27979.0,3420834.29,122.264351,239.047214,429593.63,15.354145,17

In [11]:
print(group_col.head(5))
print(group_col_coupon.head(5))

0    13
1    13
2     9
3    13
4     8
Name: campaign_id, dtype: object
0      27
1     116
2     635
3     644
4    1017
Name: coupon_id, dtype: object


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#rain[cat_cols].apply(le.fit_transform, inplace=True)
#tst[cat_cols].apply(le.fit_transform, inplace=True)
for col in test.columns.values:
       # Encoding only categorical variables
       if test[col].dtypes=='object' or test[col].dtypes=='datetime64[ns]':
       # Using whole data to form an exhaustive list of levels
           data=train[col].append(test[col])
           le.fit(data.values)
           train[col]=le.transform(train[col])
           test[col]=le.transform(test[col])

LabelEncoder()

LabelEncoder()

LabelEncoder()

In [13]:
test_ids = test[id_col]
train_ids = train[id_col]

In [14]:
y = train[target_col]
#droppping id and redemotion
train.drop(columns=[x for x in [id_col] + columns_to_drop + [target_col] if x in train.columns], inplace=True)
test.drop(columns=[x for x in [id_col] + columns_to_drop + [target_col] if x in test.columns], inplace=True)

In [15]:
print(test['end_date'].dtypes)

int64


In [16]:
train.shape
train.head()

(78369, 106)

,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_sum_mean,selling_price_sum_std,selling_price_sum_last,other_discount_sum_mean,other_discount_sum_std,other_discount_sum_last,coupon_discount_sum_mean,coupon_discount_sum_std,coupon_discount_sum_last,coupon_applied_sum_mean,coupon_applied_sum_std,coupon_applied_sum_last,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_sum_mean_cuhtd,selling_price_sum_std_cuhtd,selling_price_sum_last_cuhtd,other_discount_sum_mean_cuhtd,other_discount_sum_std_cuhtd,other_discount_sum_last_cuhtd,coupon_discount_sum_mean_cuhtd,coupon_discount_sum_std_cuhtd,coupon_discount_sum_last_cuhtd,coupon_applied_sum_mean_cuhtd,coupon_applied_sum_std_cuhtd,coupon_applied_sum_last_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_sum_mean_cucohtd,selling_price_sum_std_cucohtd,selling_price_sum_last_cucohtd,other_discount_sum_mean_cucohtd,other_discount_sum_std_cucohtd,other_discount_sum_last_cucohtd,coupon_discount_sum_mean_cucohtd,coupon_discount_sum_std_cucohtd,coupon_discount_sum_last_cucohtd,coupon_applied_sum_mean_cucohtd,coupon_applied_sum_std_cucohtd,coupon_applied_sum_last_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,13,27,1053,0,16,16,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,13,116,48,0,16,16,47,66.0,4717.27,71.473788,28.756222,677.15,10.259848,0.00,0.000000,0.0,0.000000,53.0,89.005094,45.728057,59.49,12.776415,10.914820,11.4,0.000000,0.000000,0.00,0.000000,0.000000,0.0,353,85211.51,241.392380,414.052660,9476.60,26.845892,1237.79,3.506487,12,0.033994,60,1420.191833,703.689159,124.67,157.943333,201.953340,0.00,20.629833,53.818250,0.0,0.200000,0.546173,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,27979.0,3420834.29,122.264351,239.047214,429593.63,15.354145,1702.97,0.060866,28.0,0.001001,351.0,36950.55,105.272222,129.21

Importing dependencies

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from time import time
import lightgbm as lgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

XGBoost

In [18]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

In [19]:
#gbm_param_grid = { 'colsample_bytree': np.arange(0.1,1),
#    'gamma': np.arange(0.05,1.05,.01),
#     'learning_rate' : (0.04,0.05,0.06,0.07,0.08,0.09,0.1),
#    'max_depth': (4,5,6),
#    'min_child_weight': range(1,8),
#    'n_estimators': [100,500,1000,10000],
#    'subsample':[i/100.0 for i in range(65,95,5)]}
#gbm = xgb.XGBClassifier(objective='binary:logistic')
#randomized_roc = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid,n_iter=25, scoring='roc_auc', cv=4, verbose=1)

In [20]:
#randomized_roc.fit(train, y)
#print("Best parameters found: ",randomized_roc.best_params_)
#print("Best ROC found: ", (randomized_roc.best_score_))

Best parameters found:  {'subsample': 0.9, 'n_estimators': 500, 'min_child_weight': 2, 'max_depth': 5, 'learning_rate': 0.06, 'gamma': 0.35000000000000003, 'colsample_bytree': 0.1}

Best ROC found:  0.960217967025078

In [21]:
speed={}
accuracy={}

model = XGBClassifier(subsample= 0.9, 
                      n_estimators= 500, 
                      min_child_weight= 2, 
                      max_depth= 5, 
                      learning_rate= 0.06, 
                      gamma= 0.35000000000000003, 
                      colsample_bytree= 0.1)

start=time()
cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=0)
score=cross_val_score(model,train.values, y.values,scoring="roc_auc",cv=cv,n_jobs=-1)

speed["GB"]=np.round(time() - start,3)
accuracy["GB"]=np.mean(score).round(3)

print(speed["GB"])
print(accuracy["GB"])
print(np.std(score))

45.902
0.956
0.011532920909066664


In [22]:
xgbc = model
xgbc.fit(train.values, y.values)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:07:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.1,
              gamma=0.35000000000000003, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.06, max_delta_step=0,
              max_depth=5, min_child_weight=2, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=16,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [23]:
y_pred=xgbc.predict_proba(test[train.columns].values)[:,1]

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [24]:
import joblib
# save model
joblib.dump(xgbc, 'xgbc.pkl')
# load model
#gbm_pickle = joblib.load('xgbc.pkl')

['xgbc.pkl']

In [25]:
pd.DataFrame({"id": train_ids, "redemption_status": xgbc}).to_csv('xgb-xgbc.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": y_pred}).to_csv('xgb-y-preds.csv', index=False)

LightGBM

In [26]:
#lgbm_param_grid = { 'colsample_bytree': np.arange(0.1,1),
#    'gamma': np.arange(0.05,1.05,.01),
#     'learning_rate' : np.arange(0.01,0.1),
#    'max_depth': (4,5,6),
#    'min_child_weight': range(8,80),'num_leaves': range(100,200),'subsample_freq': (4,5,6),
#    'n_estimators': [10000,20000],
#    'subsample':[i/100.0 for i in range(15,95,5)]}
#lgbm = LGBMClassifier(boosting_type= 'gbdt',objective='binary',metric='custom')
#randomized_roc = RandomizedSearchCV(estimator=lgbm, param_distributions=lgbm_param_grid,n_iter=25, scoring='roc_auc', cv=4, verbose=1)

In [27]:
#randomized_roc.fit(train, y)
#print("Best parameters found: ",randomized_roc.best_params_)
#print("Best ROC found: ", (randomized_roc.best_score_))

Best parameters found:  {'subsample_freq': 6, 'subsample': 0.85, 'num_leaves': 160, 'n_estimators': 10000, 'min_child_weight': 20, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0.14, 'colsample_bytree': 0.1}

Best ROC found:  0.9554705076478014

In [28]:
speed={}
accuracy={}

model = lgb.LGBMClassifier(subsample_freq= 6, 
                           subsample= 0.85, 
                           num_leaves= 160, 
                           n_estimators= 10000, 
                           min_child_weight= 20, 
                           max_depth= 5, 
                           learning_rate= 0.01, 
                           gamma= 0.14, 
                           colsample_bytree= 0.1)


start=time()
cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=0)
score=cross_val_score(model,train.values, y.values,scoring="roc_auc",cv=cv,n_jobs=-1)

speed["GB"]=np.round(time() - start,3)
accuracy["GB"]=np.mean(score).round(3)

print(speed["GB"])
print(accuracy["GB"])
print(np.std(score))

224.126
0.951
0.011520263727315695


In [29]:
clf = model
clf.fit(train.values, y.values)

[LightGBM] [Warning] Unknown parameter: gamma


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.1,
               gamma=0.14, importance_type='split', learning_rate=0.01,
               max_depth=5, min_child_samples=20, min_child_weight=20,
               min_split_gain=0.0, n_estimators=10000, n_jobs=-1,
               num_leaves=160, objective=None, random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=0.85,
               subsample_for_bin=200000, subsample_freq=6)

In [30]:
y_pred=clf.predict_proba(test[train.columns].values)[:,1]

In [31]:
import joblib
# save model
joblib.dump(clf, 'lgb.pkl')
# load model
#gbm_pickle = joblib.load('lgb.pkl')

['lgb.pkl']

In [32]:
pd.DataFrame({"id": train_ids, "redemption_status": clf}).to_csv('lgb-clf.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": y_pred}).to_csv('lgb-y-preds.csv', index=False)

Catboost

In [33]:
speed={}
accuracy={}

model = CatBoostClassifier()
#model=GradientBoostingClassifier()

start=time()
cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=0)
score=cross_val_score(model,train.values, y.values,scoring="roc_auc",cv=cv,n_jobs=-1)

speed["GB"]=np.round(time() - start,3)
accuracy["GB"]=np.mean(score).round(3)

print(speed["GB"])
print(accuracy["GB"])
print(np.std(score))

206.461
0.946
0.011536745419002086


In [34]:
model_cb = CatBoostClassifier()
model_cb.fit(train.values, y.values)

Learning rate set to 0.066334
0:	learn: 0.5165415	total: 90.8ms	remaining: 1m 30s
1:	learn: 0.3888685	total: 123ms	remaining: 1m 1s
2:	learn: 0.3086079	total: 154ms	remaining: 51.2s
3:	learn: 0.2448998	total: 187ms	remaining: 46.5s
4:	learn: 0.1956276	total: 222ms	remaining: 44.1s
5:	learn: 0.1599028	total: 253ms	remaining: 42s
6:	learn: 0.1293738	total: 283ms	remaining: 40.2s
7:	learn: 0.1065635	total: 314ms	remaining: 38.9s
8:	learn: 0.0898728	total: 345ms	remaining: 38s
9:	learn: 0.0772055	total: 376ms	remaining: 37.2s
10:	learn: 0.0681346	total: 410ms	remaining: 36.8s
11:	learn: 0.0608328	total: 441ms	remaining: 36.3s
12:	learn: 0.0551064	total: 475ms	remaining: 36.1s
13:	learn: 0.0508578	total: 508ms	remaining: 35.8s
14:	learn: 0.0471981	total: 539ms	remaining: 35.4s
15:	learn: 0.0446331	total: 572ms	remaining: 35.2s
16:	learn: 0.0424110	total: 605ms	remaining: 35s
17:	learn: 0.0404256	total: 638ms	remaining: 34.8s
18:	learn: 0.0389046	total: 672ms	remaining: 34.7s
19:	learn: 0.03

163:	learn: 0.0241021	total: 5.19s	remaining: 26.5s
164:	learn: 0.0240751	total: 5.22s	remaining: 26.4s
165:	learn: 0.0240529	total: 5.25s	remaining: 26.4s
166:	learn: 0.0240022	total: 5.28s	remaining: 26.3s
167:	learn: 0.0239824	total: 5.3s	remaining: 26.3s
168:	learn: 0.0239507	total: 5.33s	remaining: 26.2s
169:	learn: 0.0239208	total: 5.36s	remaining: 26.2s
170:	learn: 0.0238985	total: 5.39s	remaining: 26.1s
171:	learn: 0.0238894	total: 5.42s	remaining: 26.1s
172:	learn: 0.0238527	total: 5.45s	remaining: 26s
173:	learn: 0.0238142	total: 5.47s	remaining: 26s
174:	learn: 0.0237675	total: 5.5s	remaining: 25.9s
175:	learn: 0.0237368	total: 5.53s	remaining: 25.9s
176:	learn: 0.0236895	total: 5.56s	remaining: 25.9s
177:	learn: 0.0236487	total: 5.59s	remaining: 25.8s
178:	learn: 0.0236243	total: 5.62s	remaining: 25.8s
179:	learn: 0.0235441	total: 5.65s	remaining: 25.8s
180:	learn: 0.0234447	total: 5.68s	remaining: 25.7s
181:	learn: 0.0234225	total: 5.71s	remaining: 25.7s
182:	learn: 0.0233

325:	learn: 0.0193046	total: 9.9s	remaining: 20.5s
326:	learn: 0.0192707	total: 9.93s	remaining: 20.4s
327:	learn: 0.0192470	total: 9.96s	remaining: 20.4s
328:	learn: 0.0192352	total: 9.98s	remaining: 20.4s
329:	learn: 0.0191984	total: 10s	remaining: 20.3s
330:	learn: 0.0191685	total: 10s	remaining: 20.3s
331:	learn: 0.0191514	total: 10.1s	remaining: 20.3s
332:	learn: 0.0191101	total: 10.1s	remaining: 20.2s
333:	learn: 0.0191048	total: 10.1s	remaining: 20.2s
334:	learn: 0.0190854	total: 10.2s	remaining: 20.2s
335:	learn: 0.0190713	total: 10.2s	remaining: 20.1s
336:	learn: 0.0190371	total: 10.2s	remaining: 20.1s
337:	learn: 0.0190082	total: 10.2s	remaining: 20.1s
338:	learn: 0.0189574	total: 10.3s	remaining: 20s
339:	learn: 0.0189183	total: 10.3s	remaining: 20s
340:	learn: 0.0188834	total: 10.3s	remaining: 20s
341:	learn: 0.0188521	total: 10.4s	remaining: 19.9s
342:	learn: 0.0188480	total: 10.4s	remaining: 19.9s
343:	learn: 0.0188033	total: 10.4s	remaining: 19.9s
344:	learn: 0.0187790	t

486:	learn: 0.0159868	total: 14.5s	remaining: 15.3s
487:	learn: 0.0159694	total: 14.6s	remaining: 15.3s
488:	learn: 0.0159389	total: 14.6s	remaining: 15.3s
489:	learn: 0.0159231	total: 14.6s	remaining: 15.2s
490:	learn: 0.0158894	total: 14.7s	remaining: 15.2s
491:	learn: 0.0158802	total: 14.7s	remaining: 15.2s
492:	learn: 0.0158448	total: 14.7s	remaining: 15.1s
493:	learn: 0.0158030	total: 14.8s	remaining: 15.1s
494:	learn: 0.0157835	total: 14.8s	remaining: 15.1s
495:	learn: 0.0157592	total: 14.8s	remaining: 15s
496:	learn: 0.0157152	total: 14.8s	remaining: 15s
497:	learn: 0.0156799	total: 14.9s	remaining: 15s
498:	learn: 0.0156665	total: 14.9s	remaining: 14.9s
499:	learn: 0.0156573	total: 14.9s	remaining: 14.9s
500:	learn: 0.0156322	total: 14.9s	remaining: 14.9s
501:	learn: 0.0156032	total: 15s	remaining: 14.9s
502:	learn: 0.0155987	total: 15s	remaining: 14.8s
503:	learn: 0.0155771	total: 15s	remaining: 14.8s
504:	learn: 0.0155604	total: 15.1s	remaining: 14.8s
505:	learn: 0.0155362	to

649:	learn: 0.0132713	total: 19.4s	remaining: 10.4s
650:	learn: 0.0132649	total: 19.4s	remaining: 10.4s
651:	learn: 0.0132592	total: 19.4s	remaining: 10.4s
652:	learn: 0.0132499	total: 19.4s	remaining: 10.3s
653:	learn: 0.0132451	total: 19.5s	remaining: 10.3s
654:	learn: 0.0132324	total: 19.5s	remaining: 10.3s
655:	learn: 0.0132140	total: 19.6s	remaining: 10.3s
656:	learn: 0.0131944	total: 19.6s	remaining: 10.2s
657:	learn: 0.0131850	total: 19.6s	remaining: 10.2s
658:	learn: 0.0131819	total: 19.6s	remaining: 10.2s
659:	learn: 0.0131709	total: 19.7s	remaining: 10.1s
660:	learn: 0.0131619	total: 19.7s	remaining: 10.1s
661:	learn: 0.0131479	total: 19.7s	remaining: 10.1s
662:	learn: 0.0131192	total: 19.8s	remaining: 10s
663:	learn: 0.0130951	total: 19.8s	remaining: 10s
664:	learn: 0.0130831	total: 19.8s	remaining: 9.98s
665:	learn: 0.0130776	total: 19.8s	remaining: 9.95s
666:	learn: 0.0130666	total: 19.9s	remaining: 9.92s
667:	learn: 0.0130450	total: 19.9s	remaining: 9.89s
668:	learn: 0.01

811:	learn: 0.0112761	total: 24s	remaining: 5.56s
812:	learn: 0.0112712	total: 24s	remaining: 5.53s
813:	learn: 0.0112640	total: 24.1s	remaining: 5.5s
814:	learn: 0.0112607	total: 24.1s	remaining: 5.47s
815:	learn: 0.0112511	total: 24.1s	remaining: 5.44s
816:	learn: 0.0112362	total: 24.2s	remaining: 5.41s
817:	learn: 0.0112191	total: 24.2s	remaining: 5.38s
818:	learn: 0.0112084	total: 24.2s	remaining: 5.35s
819:	learn: 0.0111871	total: 24.3s	remaining: 5.33s
820:	learn: 0.0111836	total: 24.3s	remaining: 5.29s
821:	learn: 0.0111654	total: 24.3s	remaining: 5.26s
822:	learn: 0.0111531	total: 24.3s	remaining: 5.24s
823:	learn: 0.0111385	total: 24.4s	remaining: 5.21s
824:	learn: 0.0111202	total: 24.4s	remaining: 5.17s
825:	learn: 0.0111041	total: 24.4s	remaining: 5.15s
826:	learn: 0.0110875	total: 24.5s	remaining: 5.12s
827:	learn: 0.0110813	total: 24.5s	remaining: 5.09s
828:	learn: 0.0110649	total: 24.5s	remaining: 5.06s
829:	learn: 0.0110616	total: 24.5s	remaining: 5.03s
830:	learn: 0.011

975:	learn: 0.0095760	total: 28.7s	remaining: 705ms
976:	learn: 0.0095569	total: 28.7s	remaining: 676ms
977:	learn: 0.0095462	total: 28.7s	remaining: 646ms
978:	learn: 0.0095333	total: 28.8s	remaining: 617ms
979:	learn: 0.0095128	total: 28.8s	remaining: 588ms
980:	learn: 0.0095077	total: 28.8s	remaining: 558ms
981:	learn: 0.0095012	total: 28.8s	remaining: 529ms
982:	learn: 0.0094936	total: 28.9s	remaining: 499ms
983:	learn: 0.0094836	total: 28.9s	remaining: 470ms
984:	learn: 0.0094746	total: 28.9s	remaining: 440ms
985:	learn: 0.0094699	total: 28.9s	remaining: 411ms
986:	learn: 0.0094593	total: 29s	remaining: 382ms
987:	learn: 0.0094542	total: 29s	remaining: 352ms
988:	learn: 0.0094337	total: 29s	remaining: 323ms
989:	learn: 0.0094310	total: 29.1s	remaining: 294ms
990:	learn: 0.0094169	total: 29.1s	remaining: 264ms
991:	learn: 0.0094103	total: 29.1s	remaining: 235ms
992:	learn: 0.0094066	total: 29.1s	remaining: 205ms
993:	learn: 0.0094032	total: 29.2s	remaining: 176ms
994:	learn: 0.0093

In [35]:
y_pred=model_cb.predict_proba(test[train.columns].values)[:,1]

In [36]:
import joblib
# save model
joblib.dump(model_cb, 'cb.pkl')
# load model
#gbm_pickle = joblib.load('cb.pkl')

['cb.pkl']

In [37]:
pd.DataFrame({"id": train_ids, "redemption_status": model_cb}).to_csv('cb-model_cb.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": y_pred}).to_csv('cb-y-preds.csv', index=False)